# 02 — SQL Querying (DuckDB)

In [ ]:

!pip -q install duckdb
import duckdb, pandas as pd
con = duckdb.connect(database=':memory:')
con.execute("CREATE TABLE rents AS SELECT * FROM read_csv_auto('data/rents.csv', DATEFORMAT='%Y-%m-%d')")
con.execute("CREATE TABLE households AS SELECT * FROM read_csv_auto('data/households.csv')")
con.execute("CREATE TABLE shelters AS SELECT * FROM read_csv_auto('data/shelters.csv')")
con.execute("CREATE TABLE occupancy AS SELECT * FROM read_csv_auto('data/shelter_occupancy.csv', DATEFORMAT='%Y-%m-%d')")

q1 = con.execute("""
    SELECT county, AVG(avg_rent_usd) AS avg_rent_2024
    FROM rents
    WHERE strftime(month, '%Y') = '2024'
    GROUP BY county
    ORDER BY avg_rent_2024 DESC
""").df(); print(q1)

con.execute("""
    CREATE TABLE occ_full AS
    SELECT o.*, s.county, s.capacity, CAST(o.occupied_beds AS DOUBLE)/s.capacity AS utilization
    FROM occupancy o JOIN shelters s USING (shelter_id)
""")
q2 = con.execute("""
    SELECT county, AVG(utilization) AS avg_util_last6mo
    FROM occ_full
    WHERE date >= '2024-07-01'
    GROUP BY county
    ORDER BY avg_util_last6mo DESC
""").df(); print(q2)

q3 = con.execute("""
    SELECT county, year, AVG(eviction_filed_next_6mo) AS eviction_rate
    FROM households
    GROUP BY county, year
    ORDER BY year, eviction_rate DESC
""").df(); print(q3.head())
